In [66]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

In [67]:
## pdf docs loader
from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader
## text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
## huggingface embeddings
from langchain_huggingface import HuggingFaceEmbeddings
## vector store
from langchain_community.vectorstores import FAISS
## chatgroq
from langchain_groq import ChatGroq
## retriever
from langchain.chains import create_retrieval_chain
## combine documnets
from langchain.chains.combine_documents import create_stuff_documents_chain
## chat prompt
from langchain_core.prompts import ChatPromptTemplate

In [68]:
## api key assign
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')
os.environ['HF_TOKEN'] = os.environ['HF_TOKEN']

In [69]:
model = "Llama3-8b-8192"
llm = ChatGroq(model=model, groq_proxy=groq_api_key)
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [70]:
prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based on the provided context only.
    <context>
    {context}
    <context>
    Question: {input}
    """)

In [71]:
file_path = "/workspaces/HuggingFace/data/Attention.pdf"
loader = PyPDFLoader(file_path=file_path)
docs = loader.load()

In [72]:
## split docs
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
final_docs = splitter.split_documents(docs)

In [73]:
vectorestore = FAISS.from_documents(final_docs, embeddings)

In [74]:
question = "what is attention machenish?"

In [77]:
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectorestore.as_retriever()
retriever_chain = create_retrieval_chain(retriever, document_chain)


In [78]:
retriever_chain.invoke({"input":question})

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}